### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

/home/morrowbord/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [17]:
X_test[['description', 'company_profile', 'benefits']].head(3)

,description,company_profile,benefits
0,Stylect is a dynamic startup that helps helps ...,NaN,We are negotiable on salary and there is the p...
1,General Summary: Achieves maximum sales profit...,NaN,Great Health and DentalFast Advancement Opport...
2,"At ustwo™ you get to be yourself, whilst deliv...",ustwo offers you the opportunity to be yoursel...,NaN


In [21]:
import urllib.request
import json      

def get_prediction(description, company_profile, benefits):
    body = {'description': description, 
                            'company_profile': company_profile,
                            'benefits': benefits} 

    myurl = "http://127.0.0.1:5000/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [44]:
get_prediction('test', 'test job', '888888888 ')

0.09838830841161465

In [33]:
%%time
predictions = X_test[['description', 'company_profile', 'benefits']].iloc[:500].apply(lambda x: get_prediction(x[0], 
                                                                                               x[1], 
                                                                                               x[2]), 1)

CPU times: user 602 ms, sys: 131 ms, total: 733 ms
Wall time: 12.7 s


In [34]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525

In [35]:
#X_test[['description', 'company_profile', 'benefits']].head(3)

Давайте подумаем, где наше api может сломаться

In [36]:
get_prediction("", "", None)

0.1351884184808858